In [1]:
# attach to the same event-loop
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

# Set up the root logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Set logger level to INFO

# Clear out any existing handlers
logger.handlers = []

# Set up the StreamHandler to output to sys.stdout (Colab's output)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)  # Set handler level to INFO

# Add the handler to the logger
logger.addHandler(handler)

In [ ]:
import json
from llama_index.core import Document, VectorStoreIndex
from llama_index.core.schema import TextNode
from llama_index.llms.openai import OpenAI
from llama_index.core.evaluation import (
    DatasetGenerator,
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    CorrectnessEvaluator,
    generate_question_context_pairs,
    RetrieverEvaluator
)
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
os.chdir(os.getenv("PYTHONPATH"))

In [3]:
# import importlib
# import sys
# importlib.reload(sys.modules['src.modules'])
from src.modules import get_query_engine

/home/alvinrach/rag-aja/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
query_engine, _, chroma_collection = get_query_engine("gpt-4o-mini")

Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [5]:
response = query_engine.query("lupa pin")
print(response.response)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Jika kamu lupa PIN, kamu bisa membuat PIN baru melalui aplikasi LinkAja dengan langkah-langkah berikut:

1. Log out dari aplikasi LinkAja kamu.
2. Masukkan nomor HP dan kode OTP yang dikirim melalui SMS.
3. Pada halaman masukkan PIN, tekan tombol lupa PIN.
4. PIN sementara akan dikirimkan ke email kamu, pastikan email sudah di verifikasi sebelumnya.
5. Buka email dan ikuti petunjuk yang diberikan.
6. Buka aplikasi LinkAja kembali, ikuti instruksi login, dan masukkan 6 digit PIN baru. 

Jika kamu mengalami kesulitan, kamu juga bisa menggunakan opsi reset PIN melalui pertanyaan keamanan yang telah disetting sebelumnya.


In [5]:
gpt4 = OpenAI(model="gpt-4o-mini", temperature=0.1)

# Faithfullness

In [7]:
eval_query = "linkaja adalah"

In [8]:
faithfulness_evaluator = FaithfulnessEvaluator(llm=gpt4)
response_vector = query_engine.query(eval_query)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [9]:
eval_result = faithfulness_evaluator.evaluate_response(
    response=response_vector
)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [10]:
eval_result.model_dump()

{'query': None,
 'contexts': ['Pertanyaan: mekanisme perubahan tcash ke linkaja\nJawaban: Pelanggan TCASH dapat menjadi Pelanggan LinkAja dengan cara sebagai berikut:\n\na.Melakukan pembaharuan aplikasi di Google Play Store (Android) atau App Store (iOS); \nb. Membaca dan menyetujui syarat dan ketentuan terkait perpindahan layanan dari TCASH ke LinkAja (link SnK) dengan cara memberikan persetujuan (customer consent) dan masuk atau log in ke dalam aplikasi LinkAja menggunakan nomor handphone;\nc. Menggunakan layanan atau fasilitas yang terdapat pada LinkAja dengan melakukan transaksi pertama kali pada tanggal perpindahan layanan dengan menggunakan TAP (NFC Stiker), USSD, layanan pembayaran online melalui aplikasi merchant, serta layanan pembayaran langsung di aplikasi merchant transportasi;\nd.Selama masa perpindahan layanan, demi kenyamanan Pelanggan dan komitmen kami terhadap pemenuhan peraturan perundang-undangan yang berlaku, kami mungkin akan memintakan persetujuan Pelanggan kembal

# Relevancy

In [11]:
relevancy_evaluator = RelevancyEvaluator(llm=gpt4)

In [12]:
response_vector = query_engine.query(eval_query)

eval_result = relevancy_evaluator.evaluate_response(
    query=eval_query, response=response_vector
)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [13]:
eval_result.model_dump()

{'query': 'linkaja adalah',
 'contexts': ['Pertanyaan: mekanisme perubahan tcash ke linkaja\nJawaban: Pelanggan TCASH dapat menjadi Pelanggan LinkAja dengan cara sebagai berikut:\n\na.Melakukan pembaharuan aplikasi di Google Play Store (Android) atau App Store (iOS); \nb. Membaca dan menyetujui syarat dan ketentuan terkait perpindahan layanan dari TCASH ke LinkAja (link SnK) dengan cara memberikan persetujuan (customer consent) dan masuk atau log in ke dalam aplikasi LinkAja menggunakan nomor handphone;\nc. Menggunakan layanan atau fasilitas yang terdapat pada LinkAja dengan melakukan transaksi pertama kali pada tanggal perpindahan layanan dengan menggunakan TAP (NFC Stiker), USSD, layanan pembayaran online melalui aplikasi merchant, serta layanan pembayaran langsung di aplikasi merchant transportasi;\nd.Selama masa perpindahan layanan, demi kenyamanan Pelanggan dan komitmen kami terhadap pemenuhan peraturan perundang-undangan yang berlaku, kami mungkin akan memintakan persetujuan Pela

# Try More Data to Faithfullness and Relevancy Evaluation

In [14]:
documents = []
all_documents = chroma_collection.get()
for doc, metadata in zip(all_documents["documents"], all_documents["metadatas"]):
    documents.append(Document(text=doc, metadata=metadata))

In [15]:
dataset_generator = DatasetGenerator.from_documents(
    documents,
    llm=gpt4
)

eval_dataset = dataset_generator.generate_dataset_from_nodes(num=20)

eval_queries = list(eval_dataset.queries.values())

/home/alvinrach/rag-aja/venv/lib/python3.12/site-packages/llama_index/core/evaluation/dataset_generation.py:200: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.co

/home/alvinrach/rag-aja/venv/lib/python3.12/site-packages/llama_index/core/evaluation/dataset_generation.py:296: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [16]:
faithfulness_evaluator = FaithfulnessEvaluator(llm=gpt4)
relevancy_evaluator = RelevancyEvaluator(llm=gpt4)

faithfulness_eval_results = []
relevancy_eval_results = []
for question in eval_queries:
    response_vector = query_engine.query(question)
    faithfulness_eval_result = faithfulness_evaluator.evaluate_response(
        response=response_vector
    )
    faithfulness_eval_results.append(faithfulness_eval_result.passing)

    relevancy_eval_result = relevancy_evaluator.evaluate_response(
        query=question, response=response_vector
    )
    relevancy_eval_results.append(relevancy_eval_result.passing)


HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.co

##### Average Faithfulness Score

In [17]:
# Average Faithfullness Score
sum(faithfulness_eval_results)/len(faithfulness_eval_results)

0.75

##### Average Relevancy Score

In [18]:
# Average Relevancy Score
sum(relevancy_eval_results)/len(relevancy_eval_results)

0.85

# Correctness

In [19]:
correctness_evaluator = CorrectnessEvaluator(llm=gpt4)

In [20]:
query = "cara upgrade full service"
reference = "Hai Sobat, akun Kamu masih Basic ? Upgrade Yuuk\nUntuk mendapatkan layanan full service LinkAja, semua pelanggan Telkomsel dan non-Telkomsel  dapat melakukan eKYC melalui aplikasi LinkAja dengan menyiapkan kartu identitas asli (e-KTP) dan mengisi data diri kamu di Aplikasi LinkAja."

In [21]:
response_vector = query_engine.query(query)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [22]:
correctness_result = correctness_evaluator.evaluate(
    query=query,
    response=response_vector.response,
    reference=reference
)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [23]:
correctness_result.model_dump()

{'query': 'cara upgrade full service',
 'contexts': None,
 'response': 'Hai Sobat, akun Kamu masih Basic? Upgrade Yuuk! Untuk mendapatkan layanan full service LinkAja, semua pelanggan Telkomsel dan non-Telkomsel dapat melakukan eKYC melalui aplikasi LinkAja. Siapkan kartu identitas asli (e-KTP) dan isi data diri kamu di Aplikasi LinkAja.',
 'passing': True,
 'feedback': 'The generated answer is fully relevant and correct, matching the reference answer in both content and clarity.',
 'score': 5.0,
 'pairwise_source': None,
 'invalid_result': False,
 'invalid_reason': None}

# Try More Data to Correctness Evaluation

In [3]:
with open("../data/linkaja_pair_question_answer.json", "r") as f:
    data = json.load(f)

In [25]:
import asyncio
correctness_evaluator = CorrectnessEvaluator(llm=gpt4)

correctness_results = []
counted = 0
for i in data:
    response_vector = query_engine.query(i["question"])

    correctness_result = correctness_evaluator.evaluate(
        query=i["question"],
        response=response_vector.response,
        reference=i["answer"]
    )

    correctness_results.append(correctness_result.passing)
    counted+=1

    if not counted%10:
        await asyncio.sleep(30)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.co

##### Average Correctness Score

In [26]:
# Average Correctness Score
sum(correctness_results)/len(correctness_results)

0.9333333333333333

# Retrieval Evaluation

In [6]:
nodes = []
all_documents = chroma_collection.get()
for doc, metadata in zip(all_documents["documents"], all_documents["metadatas"]):
    metadata = {"topik": metadata["topik"], "level": metadata["level"]}
    nodes.append(TextNode(text=doc, metadata=metadata))

In [7]:
vector_index = VectorStoreIndex(nodes)

In [8]:
# [i['__data__'] for i in vector_index._docstore.to_dict()['docstore/data'].values()]

In [9]:
retriever = vector_index.as_retriever(similarity_top_k=2)
retriever_nodes = retriever.retrieve("linkaja adalah")

In [10]:
retriever_nodes

[NodeWithScore(node=TextNode(id_='5c0a5e49-2149-40c5-a576-30bbe1b388eb', embedding=None, metadata={'topik': 'aplikasi', 'level': 'linkaja'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Pertanyaan: mekanisme perubahan tcash ke linkaja\nJawaban: Pelanggan TCASH dapat menjadi Pelanggan LinkAja dengan cara sebagai berikut:\n\na.Melakukan pembaharuan aplikasi di Google Play Store (Android) atau App Store (iOS); \nb. Membaca dan menyetujui syarat dan ketentuan terkait perpindahan layanan dari TCASH ke LinkAja (link SnK) dengan cara memberikan persetujuan (customer consent) dan masuk atau log in ke dalam aplikasi LinkAja menggunakan nomor handphone;\nc. Menggunakan layanan atau fasilitas yang terdapat pada LinkAja dengan melakukan transaksi pertama kali pada tanggal perpindahan layanan dengan menggunakan TAP (NFC Stiker), USSD, layanan pembayaran online melalui aplikasi merchant, serta lay

In [11]:
from llama_index.core.response.notebook_utils import display_source_node

for node in retriever_nodes:
    display_source_node(node, source_length=2000)

**Node ID:** 5c0a5e49-2149-40c5-a576-30bbe1b388eb<br>**Similarity:** 0.5124611061179443<br>**Text:** Pertanyaan: mekanisme perubahan tcash ke linkaja
Jawaban: Pelanggan TCASH dapat menjadi Pelanggan LinkAja dengan cara sebagai berikut:

a.Melakukan pembaharuan aplikasi di Google Play Store (Android) atau App Store (iOS); 
b. Membaca dan menyetujui syarat dan ketentuan terkait perpindahan layanan dari TCASH ke LinkAja (link SnK) dengan cara memberikan persetujuan (customer consent) dan masuk atau log in ke dalam aplikasi LinkAja menggunakan nomor handphone;
c. Menggunakan layanan atau fasilitas yang terdapat pada LinkAja dengan melakukan transaksi pertama kali pada tanggal perpindahan layanan dengan menggunakan TAP (NFC Stiker), USSD, layanan pembayaran online melalui aplikasi merchant, serta layanan pembayaran langsung di aplikasi merchant transportasi;
d.Selama masa perpindahan layanan, demi kenyamanan Pelanggan dan komitmen kami terhadap pemenuhan peraturan perundang-undangan yang berlaku, kami mungkin akan memintakan persetujuan Pelanggan kembali apabila Pelanggan melakukan transaksi melalui saluran transaksi berbeda yang tersedia pada LinkAja baik melalui aplikasi, TAP (NFC Stiker), USSD, layanan pembayaran online melalui aplikasi merchant, atau layanan pembayaran langsung di aplikasi merchant transportasi. Data atas persetujuan Pelanggan melalui transaksi menggunakan LinkAja tersebut akan kami simpan dan akan kami perlakukan sesuai peraturan perundangan yang berlaku.<br>

**Node ID:** c3868bf1-abc7-4b61-bbf4-b052ce3c078e<br>**Similarity:** 0.5059245855308878<br>**Text:** Pertanyaan: apa yang terjadi dengan saldo di tcash pelanggan setelah update ke linkaja
Jawaban: Saldo Pelanggan TCASH yang telah melakukan update ke LinkAja  akan dipindahkan secara otomatis menjadi saldo di akun LinkAja dengan nilai yang sama.<br>

In [12]:
qa_dataset = generate_question_context_pairs(
    nodes, llm=gpt4, num_questions_per_chunk=2
)

  0%|          | 0/75 [00:00<?, ?it/s]

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


  1%|▏         | 1/75 [00:01<01:37,  1.31s/it]

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


  3%|▎         | 2/75 [00:02<01:10,  1.03it/s]

...

 97%|█████████▋| 73/75 [01:17<00:02,  1.26s/it]

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


 99%|█████████▊| 74/75 [01:18<00:01,  1.19s/it]

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


100%|██████████| 75/75 [01:20<00:00,  1.07s/it]


In [13]:
queries = qa_dataset.queries.values()
print(list(queries)[2])

Apa langkah pertama yang harus dilakukan oleh pelanggan LinkAja jika mereka lupa PIN lama mereka?


In [14]:
len(list(queries))

150

In [15]:
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever, llm=gpt4
)

In [16]:
sample_id, sample_query = list(qa_dataset.queries.items())[0]
sample_expected = qa_dataset.relevant_docs[sample_id]

eval_result = retriever_evaluator.evaluate(sample_query, sample_expected)

In [17]:
eval_result.model_dump()

{'query': 'Apa langkah-langkah yang harus diikuti jika pengguna lupa PIN pada aplikasi LinkAja?',
 'expected_ids': ['bc9ac7fb-d01e-4863-b8f7-aadbcf833ac8'],
 'expected_texts': None,
 'retrieved_ids': ['45ec9e20-314f-4b73-9d5c-ae4ccbb082f5',
  'fe87e771-6210-41be-8d3b-0cd43ab166cf'],
 'retrieved_texts': ['Pertanyaan: apa yang dilakukan jika lupa pin\nJawaban: Untuk pelanggan yang lupa PIN lama, bisa membuat PIN baru melalui aplikasi LinkAja. Berikut caranya:\n\n1. Log out dari aplikasi LinkAja kamu\n2. Masukkan nomor HP dan kode OTP yang dikirim melalui SMS\n3. Pada halaman masukkan PIN, tekan tombol lupa PIN\n4. PIN sementara akan dikirimkan ke email kamu, pastikan email sudah di verifikasi sebelumnya\n5. Buka email dan ikuti petunjuk\n6. Buka aplikasi LinkAja kembali, ikuti instruksi login, dan masukkan 6 digit PIN Baru',
  'Pertanyaan: biaya yang harus dikeluarkan\nJawaban: Kamu tidak akan dikenakan biaya saat download aplikasi LinkAja. Namun selayaknya aplikasi online, kamu akan dik

# Evaluate Retrieval To Whole Data

In [18]:
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

In [19]:
def display_results(name, eval_results):
    """display result from evaluate"""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"retrievers": [name], "hit_rate": [hit_rate], "mrr": [mrr]}
    )

    return metric_df, full_df

##### Retrieval Score

In [20]:
display_results("top-2 eval", eval_results)[0]

,retrievers,hit_rate,mrr
0,top-2 eval,0.56,0.5
